In [ ]:
import pandas as pd
import numpy as np
from dataclasses import dataclass
import os
import math
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary as torch_summary
import albumentations as A
import cv2
import plotly.express as px
from ultralytics import YOLO

In [ ]:
@dataclass
class Config:
    train_images_folder: str
    train_labels_folder: str
    val_images_folder: str
    val_labels_folder: str
    train_csv: str
    val_csv: str
    yolo_config_yaml: str
    training_output_folder: str
    device: str

    # noinspection PyAttributeOutsideInit
    def init(self, training):
        self.training = training
        if self.training:
            os.makedirs(self.training_output_folder, exist_ok=True)

        self.train_ids = pd.read_csv(self.train_csv)['id'].to_numpy()
        self.val_ids = pd.read_csv(self.val_csv)['id'].to_numpy()

        self.seed = 8675309
        self.batch_size = 32
        self.starting_learning_rate = 3e-4
        self.max_epochs = 40
        self.patience = 4
        self.num_workers = 8 if self.device == 'cuda' else 0
        self.pin_memory = self.num_workers > 0
        self.image_size = 640
        self.image_width = 640
        self.image_height = 384
        self.image_dims = (self.image_height, self.image_width)
        self.original_image_width = 1280
        self.original_image_height = 720
        self.use_amp = self.device == 'cuda'
        self.verbose = False

        self.imagenet_mean_cpu_tensor = torch.tensor(imagenet_mean_array)
        self.imagenet_std_cpu_tensor = torch.tensor(imagenet_std_array)
        self.channelwise_imagenet_mean_cpu_tensor = self.imagenet_mean_cpu_tensor.view(3, 1, 1)
        self.channelwise_imagenet_std_cpu_tensor = self.imagenet_std_cpu_tensor.view(3, 1, 1)
        self.imagenet_mean_gpu_tensor = gpu_tensor(imagenet_mean_array)
        self.imagenet_std_gpu_tensor = gpu_tensor(imagenet_std_array)
        self.channelwise_imagenet_mean_gpu_tensor = self.imagenet_mean_gpu_tensor.view(3, 1, 1)
        self.channelwise_imagenet_std_gpu_tensor = self.imagenet_std_gpu_tensor.view(3, 1, 1)

        self.encoder_name = 'efficientnet-b4'
        self.encoder_weights = 'imagenet'

        self.model_name = 'yolo26s.pt'

        torch.manual_seed(self.seed)
        self.generator = torch.Generator(device='cpu').manual_seed(self.seed)

        self.train_transforms = A.Compose([
            A.Resize(height=self.image_height, width=self.image_width, p=1.0),
            A.Rotate(limit=15, border_mode=cv2.BORDER_REFLECT_101, p=0.3),
            A.HorizontalFlip(p=0.5),
            A.ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.05, 0.05), p=0.7),
            A.GaussNoise(std_range=(0.0, 0.05), mean_range=(-0.02, 0.02), p=0.3),
            A.GaussianBlur(sigma_limit=(0.0, 0.5), blur_limit=0, p=0.1),
            A.Normalize(mean=imagenet_mean_tuple, std=imagenet_std_tuple),
            A.ToTensorV2(),
        ])
        self.val_transforms = A.Compose([
            A.Resize(height=self.image_height, width=self.image_width, p=1.0),
            A.Normalize(mean=imagenet_mean_tuple, std=imagenet_std_tuple),
            A.ToTensorV2(),
        ])


config: Config = None
""" Set to environment-relevant config before training/inference """;

In [ ]:
local_config = Config(
    train_images_folder='data/license_plates/train/images/',
    train_labels_folder='data/license_plates/train/labels/',
    val_images_folder='data/license_plates/val/images/',
    val_labels_folder='data/license_plates/val/labels/',
    train_csv='data/train.csv',
    val_csv='data/val.csv',
    yolo_config_yaml='data/license_plates.yaml',
    training_output_folder='data_gen/',
    device='cpu',
)

In [ ]:
imagenet_mean_tuple = (0.485, 0.456, 0.406)
imagenet_std_tuple = (0.229, 0.224, 0.225)
imagenet_mean_array = np.array([0.485, 0.456, 0.406], dtype=np.float32)
imagenet_std_array = np.array([0.229, 0.224, 0.225], dtype=np.float32)

def gpu_tensor(numpy_array):
    return torch.tensor(numpy_array, device=config.device)

def gpu_image_tensor_to_numpy_array(image_tensor):
    image = denormalize(image_tensor, config.channelwise_imagenet_mean_gpu_tensor, config.channelwise_imagenet_std_gpu_tensor)
    image = torch.clamp(image, 0, 1)
    image = image.permute(1, 2, 0).cpu().numpy()
    return (image * 255).astype(np.uint8)

def normalize(tensor, mean, std):
    return (tensor - mean) / std

def denormalize(tensor, mean, std):
    return tensor * std + mean

def print_model_torchinfo(model: nn.Module):
    print(torch_summary(model, input_size=(1, 3, config.image_width, config.image_height)))

def print_model(model: nn.Module):
    for name, module in model.named_modules():
        print(name, "->", module.__class__.__name__)

def create_dataloader(dataset, shuffle):
    return DataLoader(dataset, batch_size=config.batch_size, shuffle=shuffle, num_workers=config.num_workers, pin_memory=config.pin_memory, generator=config.generator)

def _num_batches(dataloader):
    return math.ceil(len(dataloader.dataset) / config.batch_size)

In [ ]:
def plot_val_images_with_ground_truth_bounding_boxes(num_images_to_show = 3):
    ids = np.random.choice(config.val_ids, num_images_to_show)
    for image_id in ids:
        image_path = f'{config.val_images_folder}{image_id}.jpg'
        image_label = f'{config.val_labels_folder}{image_id}.txt'
        image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        boxes = []
        with open(image_label, 'r') as label_file:
            for line in label_file:
                coords = [float(c) for c in line.strip().split()[-4:]]
                boxes.append(coords)
        fig = px.imshow(image)
        for (x_min, y_min, x_max, y_max) in boxes:
            fig.add_shape(type='rect', x0=x_min, y0=y_min, x1=x_max, y1=y_max, line_color='orange')
        fig.show()

In [ ]:
config = local_config
config.init(training=False)
plot_val_images_with_ground_truth_bounding_boxes()

In [ ]:
def train_yolo():
    model = YOLO(config.model_name)
    results = model.train(
        data=config.yolo_config_yaml,
        epochs=config.max_epochs,
        imgsz=config.image_size,
    )